In [49]:
import pandas as pd
import math

In [50]:
#Read two files and merge
df1 = pd.read_csv('data/ipo_list', sep='\t', index_col='code')
df2 = pd.read_csv('data/ipo_details', sep= '\t', index_col='code')

#Use combine_first to avoid duplicate columns
#result = pd.concat([df, df2], axis = 1, join='inner')
df = df1.combine_first(df2)

#Save to file
df.to_csv('data/ipo_data_processed.csv', sep=',')
df.columns.values

array(['acc_chg', 'applied_lots_for_one_lot', 'chg_on_debut', 'date',
       'gray_market', 'industry', 'last', 'listing_price', 'lot_size',
       'market_cap', 'name', 'offer_price', 'one_lot_success_rate',
       'over_sub_rate', 'sponsor'], dtype=object)

In [51]:
df["acc_chg"]

code
00150.HK     -2.353%
00301.HK    -22.308%
00302.HK    +30.389%
00302.HK    +30.389%
00368.HK      0.000%
              ...   
09996.HK    +76.107%
09997.HK    +72.190%
09998.HK    -55.385%
09998.HK    -55.385%
09999.HK    +24.959%
Name: acc_chg, Length: 875, dtype: object

In [52]:
#Now use dataframe and do some feature engineering
# Drop unrelated columns
to_del = ['acc_chg', 'gray_market', 'last', 'listing_price', 'date', 'name']
for item in to_del:
    del df[item]
    
#Drop non_public ipo stocks
df = df[df.one_lot_success_rate.notnull()]

In [53]:
def one_lot(x):
    return float(str(x).strip("手"))

def str2float(x):
    return float(str(x).replace(',', ''))

def buy_ratio_process(x):
    if x == '认购不足':
        return 0.0
    else:
        return float(str(x).replace(',', ''))

def hk_portion_process(x):
    if not pd.isnull(x):
        x = x.split('(')[1].strip(')')
        return x.strip()
    else:
        return x
    
def per2float(x):
    if not pd.isnull(x):
        x = x.strip('%')
        return float(x)/100.
    else:
        return x


#定价下限
def get_low_bound(x):
    if ',' in str(x):
        x = x.replace(',', '')
    try:
        if pd.isnull(x) or '-' not in x:
            return float(x)
        else:
            x = x.split('-')
            return float(x[0])
    except Exception as e:
        print(e)
        print(x)


def get_up_bound(x):
    """
    定价上限
    :param x: 
    :return: 
    """
    if ',' in str(x):
        x = x.replace(',', '')
    try:
        if pd.isnull(x) or '-' not in x:
            return float(x)
        else:
            x = x.split('-')
            return float(x[1])
    except Exception as e:
        print(e)
        print(x)

        
def get_ipo_range_prop(x):
    """
    定价范围
    :param x: 
    :return: 
    """
    if pd.isnull(x):
        return x
    low_bound = get_low_bound(x)
    up_bound = get_up_bound(x)
    return (up_bound-low_bound)*2/(up_bound+low_bound)

def get_total_value_mid(x):
    """
    中间定价
    :param x: 
    :return: 
    """
    if pd.isnull(x):
        return x
    low_bound = get_low_bound(x)
    up_bound = get_up_bound(x)
    return (up_bound+low_bound)/2
    
#Format buy_ratio
df['over_sub_rate'] = df['over_sub_rate'].apply(buy_ratio_process)

#Format hk-porrtion
#df['hk_portion'] = df['hk_portion'].apply(hk_portion_process)

#Format percentage
df['one_lot_success_rate'] = df['one_lot_success_rate'].apply(per2float)
df['chg_on_debut'] = df['chg_on_debut'].apply(per2float)
df['applied_lots_for_one_lot'] = df['applied_lots_for_one_lot'].apply(one_lot)
#df['one_lot_success_rate'] = df['one_lot_success_rate'].apply(per2float)

df['lot_size'] = df['lot_size'].apply(str2float)

#Split range to two columns
#df['ipo_price_low_bound'] = df['ipo_price_range'].apply(get_low_bound)
#df['ipo_price_up_bound'] = df['ipo_price_range'].apply(get_up_bound)
#del df['ipo_price_range']
#df['total_value_low_bound'] = df['total_value'].apply(get_low_bound)/1000000000.
#df['total_value_up_bound'] = df['total_value'].apply(get_up_bound)/1000000000.
#del df['total_value']


#Merge ipo_price_range to proportion of middle
df['ipo_price_range_ratio'] = df['offer_price'].apply(get_ipo_range_prop)
del df['offer_price']
df['total_value_mid'] = df['market_cap'].apply(get_total_value_mid)/1000000000.
del df['market_cap']


df['ipo_price_range_ratio']

code
00301.HK    0.153846
00302.HK    0.254980
00302.HK    0.254980
00368.HK    0.090909
00368.HK    0.090909
              ...   
09996.HK    0.000000
09997.HK    0.115854
09998.HK    0.142857
09998.HK    0.142857
09999.HK    0.000000
Name: ipo_price_range_ratio, Length: 806, dtype: float64

In [54]:
#Now do one-hot encoding for all categorical columns
#One problem is that we have to split('、') first for contents with multiple companies

dftest = df.copy()

def one_hot_encoding(df, column_name):
#Reads a df and target column, does tailored one-hot encoding, and return new df for merge

    cat_list = df[column_name].unique().tolist()
    cat_set = set()
    for items in cat_list:
        if pd.isnull(items):
            continue
        items = items.split('、')
        for item in items:
            item = item.strip()
            cat_set.add(item)
    for item in cat_set:
        item = column_name + '_' + item
        df[item] = 0
    
    def check_onehot(x, cat):
        if pd.isnull(x):
            return 0
        x = x.split('、')
        for item in x:
            if cat == item.strip():
                return 1
        return 0
    
    for item in cat_set:
        df[column_name + '_' + item] = df[column_name].apply(check_onehot, args=(item, ))
    
    del df[column_name]
    return df
    
dftest = one_hot_encoding(dftest, 'industry')
dftest = one_hot_encoding(dftest, 'sponsor')

dftest.to_csv('data/hk_ipo_feature_engineered.csv', sep='\t', encoding='utf-8')
dftest


,applied_lots_for_one_lot,chg_on_debut,lot_size,one_lot_success_rate,over_sub_rate,ipo_price_range_ratio,total_value_mid,industry_印刷及包装,industry_基建,industry_商用交通工具,...,sponsor_民银资本有限公司,sponsor_招银国际融资有限公司,sponsor_德意志证券亚洲有限公司,sponsor_星展亚洲融资有限公司,sponsor_华邦融资有限公司,sponsor_天泰金融服务有限公司,sponsor_Merrill Lynch Far East Limited,sponsor_安信融资(香港)有限公司,sponsor_中国银河国际证券(香港)有限公司,sponsor_中信里昂证券资本市场有限公司
code,,,,,,,,,,,,,,,,,,,,,
00301.HK,10.0,-0.20769,2000.0,0.300,36.8,0.153846,5.560000e-09,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00302.HK,20.0,0.02120,2000.0,0.150,542.8,0.254980,5.848500e-08,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00302.HK,20.0,0.02120,2000.0,0.150,542.8,0.254980,5.848500e-08,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00368.HK,25.0,0.11111,4000.0,0.100,9.7,0.090909,5.280000e-09,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00368.HK,25.0,0.11111,4000.0,0.100,9.7,0.090909,5.280000e-09,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
09996.HK,200.0,0.67969,1000.0,0.085,1183.4,0.000000,9.370000e-08,0,0,0,...,0,0,0,0,0,0,0,0,0,0
09997.HK,90.0,0.98847,500.0,0.090,988.2,0.115854,1.642900e-07,0,0,0,...,0,0,0,0,0,0,1,0,0,1
09998.HK,20.0,-0.07692,5000.0,0.053,47.7,0.142857,5.600000e-09,0,0,0,...,0,0,0,0,0,0,0,0,0,0
